In [15]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import re
from matplotlib.pyplot import figure, show, axes, sci
from matplotlib import cm, colors
from matplotlib.font_manager import FontProperties
from numpy import amin, amax, ravel
import numpy.ma as ma

In [16]:
mass_e = np.zeros( (3,3,12) )
mass_h = np.zeros( (3,3,12) )
eg = np.zeros( (3,3,12) )
data = open('/Users/yao/Google Drive/data/216/mass_paper','r')
data_eg = open('/Users/yao/Google Drive/data/216/Eg_paper','r')

In [17]:
value_e = []
value_h = []
lines = data.readlines()
for line in lines:
    match = re.match(r"([a-z]+)([ ]+)([a-z]+)([ ]+)([a-z]+)([ ]+)([0-9.]+)([ ]+)([0-9.]+)", line, re.I)
    if match:
        items = match.groups()
        value_e.append(float(items[6]))
        value_h.append(float(items[8]))

In [18]:
value_eg = []
lines = data_eg.readlines()
for line in lines:
    match = re.match(r"([a-z]+)([ ]+)([a-z]+)([ ]+)([a-z]+)([ ]+)([0-9.]+)", line, re.I)
    if match:
        items = match.groups()
        value_eg.append(float(items[6]))

In [19]:
#for i in range(len(value_e)):
 #   if value_e[i] >= 10.0:
  #      value_e[i] = 10.0
#for i in range(len(value_h)):
#    if value_h[i] >= 10.0:
 #       value_h[i] = 10.0


In [20]:
index = 0
for a in range(3):
    for b in range(12):
        for c in range(3):
            mass_e[c][a][b] = value_e[index]
            mass_h[c][a][b] = value_h[index]
            index = index + 1


In [21]:
index = 0
for a in range(3):
    for b in range(12):
        for c in range(3):
            eg[c][a][b] = value_eg[index]
            index = index + 1
    

In [22]:
masked_e = ma.masked_where(eg<0.15, mass_e)
masked_h = ma.masked_where(eg<0.15, mass_h)

In [23]:
fig = figure()
#cmap = cm.cool
w = 1.2
h = 0.15
ax = []
images = []
vmin = 1e40
vmax = -1e40
cax = fig.add_axes([0.8, 0.2, 0.02, 0.5])

In [24]:
for i in range(3):
    pos = [0, 0.56 - i*1.2*h, w, h]
    a = fig.add_axes(pos)
    plt.yticks(np.arange(3),['K','Rb','Cs'], size='x-small')
    plt.xticks(np.arange(12),[])
    if i == 2:
        #plt.xticks(np.arange(33),['Ge', 'Pd', 'Ce', 'Pr', 'Tb','Si','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Se','Zr','Nb','Mo','Tc','Ru','Rh','Pb','Ag','Sn','Te','Hf','Ta','W','Re','Os','Ir','Pt','Hg'],size='x-small')
        plt.xticks(np.arange(12),['Si','Ge','Sn','Pb','Se','Te','Ni','Pd','Pt','Mn','Tc','Re'],size='x-small')  
    dd = ravel(masked_h[i])
    # Manually find the min and max of all colors for
    # use in setting the color scale.
    vmin = min(vmin, amin(dd))
    vmax = max(vmax, amax(dd))
    images.append(a.imshow(masked_h[i],interpolation = 'nearest',aspect=2))

    ax.append(a)

In [25]:
# Set the first image as the master, with all the others
# observing it for changes in cmap or norm.

class ImageFollower:
    'update image in response to changes in clim or cmap on another image'
    def __init__(self, follower):
        self.follower = follower
    def __call__(self, leader):
        self.follower.set_cmap(leader.get_cmap())
        self.follower.set_clim(leader.get_clim())
norm = colors.LogNorm(vmin=1e-1, vmax=1e1)
for i, im in enumerate(images):
    im.set_norm(norm)
    if i > 0:
        images[0].callbacksSM.connect('changed', ImageFollower(im))
fig.colorbar(images[0], cax)
axes(ax[0])     # Return the current axes to the first one,
sci(images[0])  # because the current image must be in current axes.
show()